# Starfish BaristaSeq Processing Example

In [1]:
%gui qt

import os
from copy import deepcopy
from itertools import product

import numpy as np
import pandas as pd
import skimage.filters
import skimage.morphology
from skimage.transform import SimilarityTransform, warp
from tqdm import tqdm

import starfish
import starfish.data
from starfish.spots import SpotFinder
from starfish.types import Axes

BaristaSeq is an assay that sequences padlock-probe initiated rolling circle amplified spots using a one-hot codebook. The publication for this assay can be found [here](https://www.ncbi.nlm.nih.gov/pubmed/29190363).

here we select data for a single field of view. 

In [2]:
exp = starfish.Experiment.from_json(os.path.expanduser("~/scratch/baristaseq/experiment.json"))
nissl = exp['fov_000'].get_image('dots')

100%|██████████| 17/17 [00:00<00:00, 61.75it/s]


In [3]:
# exp = starfish.data.BaristaSeq()
img = exp['fov_000'].get_image('primary')

100%|██████████| 476/476 [00:07<00:00, 61.16it/s]


The first step in BaristaSeq is to do some rough registration. For this data, the rough registration has been done for us by the authors, so it is omitted from this notebook.

## Project into 2D

First, project the z-plane to do analysis of BaristaSeq in 2-d. 

In [4]:
z_projected_image = img.max_proj(Axes.ZPLANE)
z_projected_nissl = nissl.max_proj(Axes.ZPLANE)

100%|██████████| 1/1 [00:00<00:00, 195.01it/s]


## Correct Channel Misalignment

There is a slight miss-alignment of the C channel in the microscope used to process the data. This has been corrected for this data, but here is how it could be transformed using python code for future datasets.

In [5]:
# from skimage.feature import register_translation
# from skimage.transform import warp
# from skimage.transform import SimilarityTransform
# from functools import partial

# # Define the translation
# transform = SimilarityTransform(translation=(1.9, -0.4))

# # C is channel 0
# channels = (0,)

# # The channel should be transformed in all rounds
# rounds = np.arange(img.num_rounds)

# # apply the transformation in place
# slice_indices = product(channels, rounds)
# for ch, round_, in slice_indices:
#     selector = {Axes.ROUND: round_, Axes.CH: ch, Axes.ZPLANE: 0}
#     tile = z_projected_image.get_slice(selector)[0]
#     transformed = warp(tile, transform)
#     z_projected_image.set_slice(
#         selector=selector,
#         data=transformed.astype(np.float32),
#     )

## Remove Registration Artefacts

There are some minor registration errors along the pixels for which y < 100 and x < 50. Those pixels are dropped from this analysis

In [6]:
registration_corrected = z_projected_image.sel({Axes.Y: (100, -1), Axes.X: (50, -1)})

## Correct for bleed-through from Illumina SBS reagents

The following matrix contains bleed correction factors for Illumina sequencing-by-synthesis reagents. Starfish provides a LinearUnmixing method that will unmix the fluorescence intensities

In [7]:
data = np.array(
    [[0.  , 0.05, 0.  , 0.  ],
     [0.35, 0.  , 0.  , 0.  ],
     [0.  , 0.02, 0.  , 0.84],
     [0.  , 0.  , 0.05, 0.  ]]
)
rows = pd.Index(np.arange(4), name='bleed_from')
cols = pd.Index(np.arange(4), name='bleed_to')
unmixing_coeff = pd.DataFrame(data, rows, cols)

# show results
unmixing_coeff

bleed_to,0,1,2,3
bleed_from,,,,
0,0.00,0.05,0.00,0.00
1,0.35,0.00,0.00,0.00
2,0.00,0.02,0.00,0.84
3,0.00,0.00,0.05,0.00


In [8]:
lum = starfish.image._filter.linear_unmixing.LinearUnmixing(unmixing_coeff)
bleed_corrected = lum.run(registration_corrected)

## Remove image background

To remove image background, BaristaSeq uses a White Tophat filter, which measures the background with a rolling disk morphological element and subtracts it from the image. 

In [9]:
from skimage.morphology import opening, dilation, disk
from functools import partial

If desired, the background that is being subtracted can be visualized

In [10]:
# opening = partial(opening, selem=disk(5))

# background = bleed_corrected.apply(
#     opening,
#     group_by={Axes.ROUND, Axes.CH, Axes.ZPLANE}, verbose=False, in_place=False
# )

# starfish.display(background)

In [11]:
wth = starfish.image.Filter.WhiteTophat(masking_radius=5)
background_corrected = wth.run(bleed_corrected, in_place=False)

## Scale images to equalize spot intensities across channels

The number of peaks are not uniform across rounds and channels, which prevents histogram matching across channels. Instead, a percentile value is identified and set as the maximum across channels, and the dynamic range is extended to equalize the channel intensities

In [12]:
sbp = starfish.image.Filter.ScaleByPercentile(p=99.5)
scaled = sbp.run(background_corrected, n_processes=1, in_place=False)

In [13]:
starfish.display(scaled)

## Remove residual background

The background is fairly uniformly present below intensity=0.5. However, starfish's clip method currently only supports percentiles. To solve this problem, the intensities can be directly edited in the underlying numpy array. 

In [14]:
from copy import deepcopy
clipped = deepcopy(scaled)
clipped.xarray.values[clipped.xarray.values < 0.5] = 0

In [15]:
starfish.display(clipped)

## Detect Spots

Detect spots with a local search blob detector that identifies spots in all rounds and channels and matches them using a local search method. The local search starts in an anchor channel (default ch=1) and identifies the nearest spot in all subsequent imaging rounds. 

In [16]:
threshold = 0.5

lsbd = starfish.spots._detector.local_search_blob_detector.LocalSearchBlobDetector(
    min_sigma=(0.5, 0.5, 0.5),
    max_sigma=(8, 8, 8),
    num_sigma=10,
    threshold=threshold,
    search_radius=7
)
intensities = lsbd.run(clipped)
decoded = exp.codebook.decode_per_round_max(intensities.fillna(0))

In [17]:
starfish.display(clipped, intensities)

Based on visual inspection, it looks like the spot correspondence across rounds isn't being detected well. Try the PixelSpotDecoder.

In [26]:
psd = starfish.spots.PixelSpotDecoder.PixelSpotDecoder(
    codebook=exp.codebook, metric='euclidean', distance_threshold=0.5, 
    magnitude_threshold=0.1, min_area=7, max_area=50
)
pixel_decoded, ccdr = psd.run(clipped)

100%|██████████| 1698/1698 [00:03<00:00, 500.74it/s]


In [27]:
label_image = starfish.ImageStack.from_numpy_array(np.reshape(ccdr.label_image, (1, 1, 1, 1092, 862)))
starfish.display(label_image)

/Users/ajc/projects/spacetx/starfish/starfish/imagestack/imagestack.py:324: UserWarning: ImageStack detected as int64. Converting to float32...
  warnings.warn(f"ImageStack detected as {array.dtype}. Converting to float32...")
/Users/ajc/projects/spacetx/starfish/.venv/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from int64 to float32
  .format(dtypeobj_in, dtypeobj_out))
100%|██████████| 1/1 [00:00<00:00, 167.00it/s]


Compare the number of spots being detected by the two spot finders

In [28]:
print("pixel_decoder spots detected", int(np.sum(pixel_decoded['target'] != 'nan')))
print("local search spot detector spots detected", int(np.sum(decoded['target'] != 'nan')))

pixel_decoder spots detected 1618
local search spot detector spots detected 1454


Report the correlation between the two methods

In [29]:
from scipy.stats import pearsonr

# get the total counts for each gene from each spot detector
pixel_decoded_gene_counts = pd.Series(*np.unique(pixel_decoded['target'], return_counts=True)[::-1])
decoded_gene_counts = pd.Series(*np.unique(decoded['target'], return_counts=True)[::-1])

# get the genes that are detected by both spot finders
codetected = pixel_decoded_gene_counts.index.intersection(decoded_gene_counts.index)

# report the correlation
pearsonr(pixel_decoded_gene_counts[codetected], decoded_gene_counts[codetected])

(0.1868014722522533, 0.20361878324966726)

The pixel based spot detector looks better upon visual inspection. Do the below values make sense for this tissue and this probeset?? 

In [30]:
pixel_decoded_gene_counts.sort_values()

Nxph1         1
Neurod6       1
Pdgfra        1
Ndnf          1
Penk          1
Plpp4         1
Cnr1          1
Dlx1          1
Dcn           1
Pcp4          2
Itpka         2
Fgfr3         2
Ankrd55       2
Snrpn         2
Kcnmb2        3
Nxph4         3
Nrn1          3
Rprml         3
Id2           3
Pvalb         3
Tesc          4
Rorb          4
Sncg          4
Cxcl14        4
Fezf2         4
Sparcl1       5
Caln1         6
Arx           6
Npy           6
Reln          6
Slc6a1        7
Nfib          7
Slc17a7       9
Sst          11
Kcnip1       11
Vxn          17
Rab3b        18
Sv2b         19
Cacna2d3     24
Dlx6         24
Crh          27
Ly6c2        28
Nrgn         32
Cck          35
Ptprd        42
Car4         44
Synpr        51
Neurod1      54
Igfbp6       59
nan          80
Arpp19      134
Brinp3      135
Ptn         195
Ctxn1       549
dtype: int64

In [43]:
exp.codebook[np.where(exp.codebook["target"] == "Ctxn1")]

<xarray.Codebook (target: 1, c: 4, r: 7)>
array([[[1, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 0, 0, 0, 0]]], dtype=uint8)
Coordinates:
  * target   (target) object 'Ctxn1'
  * c        (c) int64 0 1 2 3
  * r        (r) int64 0 1 2 3 4 5 6

In [44]:
exp.codebook[np.where(exp.codebook["target"] == "Ptn")]

<xarray.Codebook (target: 1, c: 4, r: 7)>
array([[[0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [1, 1, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0]]], dtype=uint8)
Coordinates:
  * target   (target) object 'Ptn'
  * c        (c) int64 0 1 2 3
  * r        (r) int64 0 1 2 3 4 5 6

In [45]:
exp.codebook[np.where(exp.codebook["target"] == "Brinp3")]

<xarray.Codebook (target: 1, c: 4, r: 7)>
array([[[0, 1, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 1]]], dtype=uint8)
Coordinates:
  * target   (target) object 'Brinp3'
  * c        (c) int64 0 1 2 3
  * r        (r) int64 0 1 2 3 4 5 6

Looks like the codebook targets from PixelSpotDecoding don't share much in the way of channel biases across rounds or across codes, which makes me reasonably confident in the decoding result. 